In [1]:
import itertools
from pathlib import Path

import bottleneck as bn
import nicpolpy as nic
import numpy as np
import pandas as pd
import scikit_posthocs as sp
import ysfitsutilpy as yfu
import ysphotutilpy as ypu
import ysvisutilpy as yvu
from astropy import units
from astropy.nddata import CCDData
from astropy.stats import sigma_clip, sigma_clipped_stats
from astropy.time import Time
from matplotlib import pyplot as plt
from matplotlib import rcParams
from photutils import CircularAnnulus, CircularAperture, EllipticalAperture
from scipy.ndimage import gaussian_filter

%matplotlib widget

# We need to do it in a separate cell. See:
# https://github.com/jupyter/notebook/issues/3385
plt.style.use('default')
rcParams.update({
    'font.family': 'go mono, juliamono, courier',
    'font.size': 10, 'mathtext.fontset': 'stix',
    'axes.formatter.use_mathtext': True, 'axes.formatter.limits': (-4, 4),
    # 'axes.grid': True, 'grid.color': 'gray', 'grid.linewidth': 0.5,
    'xtick.top': True, 'ytick.right': True,
    'xtick.direction': 'inout', 'ytick.direction': 'inout',
    'xtick.minor.size': 4.0, 'ytick.minor.size': 4.0,  # default 2.0
    'xtick.major.size': 8.0, 'ytick.major.size': 8.0,  # default 3.5
    'xtick.minor.visible': True, 'ytick.minor.visible': True
})


def calcq(a, b, c, d):
    sqbc = np.sqrt(b*c)
    sqad = np.sqrt(a*d)
    return 100*(sqbc - sqad) / (sqbc + sqad)


# def find_rap(img, pos, r_init, r_in, r_out, error, gsigma=6, const=False):
#     ap, an = ypu.circ_ap_an(pos, r_ap=r_init, r_in=r_in, r_out=r_out)
#     phot = ypu.apphot_annulus(img, ap, an, error=error)
#     if const:
#         return r_init, phot, ap, an

#     r = min(1.*max(5, np.sqrt(phot["snr"].iloc[0])), 50)
#     if np.abs(r - r_init) < 1:
#         return r, phot, ap, an
#     else:
#         return find_rap(img, pos, r, r_in, r_out, error, const=const)


def find_pos(img, thresh_ksig, minarea=50, gsigma=6,
             box_size=(50, 50), filter_size=(5, 5)):
    obj, segm, imgdetect = nic.quick_detect_obj(img, thresh_ksig=thresh_ksig,
                                                minarea=minarea, gsigma=gsigma,
                                                scale_maxiters=10,
                                                box_size=box_size, filter_size=filter_size)
    if obj is None:
        return np.array(img.shape[::-1])/2, None, None, None
    obj = obj.loc[obj["flux"] == obj["flux"].max()].iloc[0]
    pos = np.array([obj["x"], obj["y"]])

    pos = ypu.find_centroid(img, pos, cbox_size=10, csigma=3, ssky=None, max_shift=10, max_shift_step=1)
    return pos, obj, segm, imgdetect


def find_rsnr(img, pos, sky_an, err=None, rvals=np.arange(5, 60), const=False, full=True):
    if const:
        return const, None

    import warnings
    warnings.filterwarnings('ignore', append=True, category=RuntimeWarning)

    # aps = [CircularAperture(pos, r=_r) for _r in rvals]
    # phot_all = ypu.apphot_annulus(img, aps, sky_an, error=err)
    # phot_all["snr2"] = phot_all["source_sum"]/np.sqrt(np.pi*rvals**2*(phot_all["ssky"]**2 + 3.7**2))
    # rdn = 3.7 adu for all filt
    # maxsnr = phot_all["snr2"].max()
    # if maxsnr > 0:
    #     phot = phot_all.loc[phot_all["snr2"] == maxsnr].copy()
    #     r_snr = rvals[phot.index[0]]
    #     return r_snr, phot
    # else:
    #     return rvals.min(), None
    di = [ypu.sky_fit(img, CircularAnnulus(pos, r_in=_r-0.5, r_out=_r+0.5))["msky"]
          for _r in rvals]
    sky = ypu.sky_fit(img, sky_an)
    try:
        r_ap = rvals[np.where(di > sky["msky"] + 1*sky["ssky"])[0].max()]
        if full:
            return r_ap, ypu.apphot_annulus(img, CircularAperture(pos, r=r_ap), sky_an, error=err)
        else:
            return r_ap
    except ValueError:
        if full:
            return rvals.min(), None
        else:
            return rvals.min()


def do_phot(df_o_f, setid, d4d, rapmodes, objname, filt,
            gain=1, rdn=0, flat_err=0.02, thresh_ksig=10, r_in=70, r_out=90):
    def _rap_oe(im_acbd, pos_acbd, err_acbd, r_in, r_out):
        rap_o, rap_e = {}, {}
        rsnrs = [find_rsnr(im, pos[0], sky_an=CircularAnnulus(pos[0], r_in, r_out), err=err, const=False, full=False)
                 for im, pos, err in zip(im_acbd, pos_acbd, err_acbd)]
        for rapmode in rapmodes:
            if rapmode == "max":
                rap_o[rapmode] = np.max(rsnrs)
                rap_e[rapmode] = rap_o[rapmode]
            elif rapmode == "min":
                rap_o[rapmode] = np.min(rsnrs)
                rap_e[rapmode] = rap_o[rapmode]
            elif rapmode == "maxoe":
                rap_o[rapmode] = max(rsnrs[:2])
                rap_e[rapmode] = max(rsnrs[2:])
            elif rapmode == "minoe":
                rap_o[rapmode] = min(rsnrs[:2])
                rap_e[rapmode] = min(rsnrs[2:])
            else:
                rap_o[rapmode] = rapmode
                rap_e[rapmode] = rapmode
        print([res for res in rsnrs])
        return rap_o, rap_e

    pas = []
    acbds = {_mode: dict() for _mode in rapmodes}
    dacbds = {_mode: dict() for _mode in rapmodes}

    for qu in "qu":
        _df = df_o_f.loc[df_o_f["SETID"] == f"{qu}{setid:03d}"].copy()
        # order = HWP1 o, HWP2 o, HWP1 e, HWP2 e (a, c, b, d)
        im_acbd = yfu.load_ccds(list(_df["file"]), ccddata=False, trimsec=10)
        err_acbd = [yfu.errormap(im, gain_epadu=gain, rdnoise_electron=rdn, flat_err=flat_err) for im in im_acbd]
        pos_acbd = [find_pos(im, thresh_ksig=thresh_ksig) for im in im_acbd]
        rap_o, rap_e = _rap_oe(im_acbd, pos_acbd, err_acbd, r_in, r_out)

        for rapmode in rapmodes:
            acbd, dacbd = [], []
            snrs, raps, poss = [], [], []
            aps, ans = [], []
            uts = []
            for k, (_, row) in enumerate(_df.iterrows()):
                pos = pos_acbd[k][0]
                r_ap = rap_o[rapmode] if k < 2 else rap_e[rapmode]
                ap, an = ypu.circ_ap_an(pos, r_ap, r_in=90, r_out=110)
                phot = ypu.apphot_annulus(im_acbd[k], ap, an, error=err_acbd[k])

                aps.append(ap)
                ans.append(an)
                raps.append(r_ap)
                poss.append([float(f"{_p:.3f}") for _p in pos])
                acbd.append(phot["source_sum"].iloc[0])
                dacbd.append(phot["source_sum_err"].iloc[0])
                snrs.append(phot["snr"].iloc[0])
                pas.append(row["PA"])
                uts.append((Time(row["DATE-OBS"], format="isot") + row["EXPTIME"]*units.s/2).jd)

                # print(objname, filt, qu, i, "acbd"[k], [float(f"{_p:.3f}") for _p in pos],
                #       rapmode, phot["snr"][0], r_ap)

            dtau = 0.5*np.log((acbd[0]*acbd[3])/(acbd[1]*acbd[2]))
            d4d[rapmode][f"dtau{qu}"].append(dtau)
            d4d[rapmode][f"snr_{qu}"].append(np.min(snrs))
            d4d[rapmode][f"dsnr_{qu}"].append(np.ptp(snrs))
            d4d[rapmode][f"raps_{qu}"].append(raps)
            d4d[rapmode][f"pos_{qu}"].append(poss)
            acbds[rapmode][qu] = acbd
            dacbds[rapmode][qu] = dacbd

    for rapmode in rapmodes:
        i_q = acbds[rapmode]["q"]
        i_u = acbds[rapmode]["u"]
        di_q = dacbds[rapmode]["q"]
        di_u = dacbds[rapmode]["u"]
        q, u, dq, du = ypu.calc_qu_4set(o_000=i_q[0], e_000=i_q[2], o_450=i_q[1], e_450=i_q[3],
                                        o_225=i_u[0], e_225=i_u[2], o_675=i_u[1], e_675=i_u[3],
                                        do_000=di_q[0], de_000=di_q[2], do_450=di_q[1], de_450=di_q[3],
                                        do_225=di_u[0], de_225=di_u[2], do_675=di_u[1], de_675=di_u[3],
                                        out_pct=True)

        d4d[rapmode]["setid"].append(i)
        d4d[rapmode]["filt"].append(filt)
        d4d[rapmode]["object"].append(objname)
        d4d[rapmode]["midut"].append(Time(np.mean(uts), format="jd").isot)
        d4d[rapmode]["pa"].append(np.mean(pas))
        d4d[rapmode]["q"].append(q)
        d4d[rapmode]["u"].append(u)
        d4d[rapmode]["dq"].append(dq)
        d4d[rapmode]["du"].append(du)


        
def _gesd(x, alpha=0.01, outliers=0.1, outliers_min=1):
    if (nx := x.compressed().size) >= 4:
        outliers = int(outliers) if outliers >= 1 else int(nx*outliers)
        mask = sp.outliers_gesd(x.filled(100), alpha=alpha, hypo=True,
                                outliers=max(outliers_min, outliers))
        if mask.dtype == "bool":
            x.mask = mask


def _wvg(x, dx):
    x = np.ma.array(x)
    dx = np.ma.array(dx)
    return (np.ma.average(x.compressed(), weights=1/dx.compressed()**2),
            1/np.sqrt(np.ma.sum(1/dx.compressed()**2)))


def qupa(qv, uv, dqv, duv, pa_obs, alpha=0.01, outliers=0.1, outliers_min=1):
    # def _mask_minmax(x):
    #     if x.compressed().size > 10:
    #         x.mask = (x.min() == x) | (x == x.max())  # mask min/max
    # def _sigc(x, sigma=5, stdfunc="mad_std"):
    #     if x.compressed().size > 5:
    #         _, _md, _sd = sigma_clipped_stats(x.compressed(), sigma=3, stdfunc=stdfunc)
    #         x.mask |= (x < _md - sigma*_sd) | (x > _md + sigma*_sd)

    qv = np.ma.array(qv, mask=np.isnan(qv))
    uv = np.ma.array(uv, mask=np.isnan(uv))
    dqv = np.ma.array(dqv, mask=np.isnan(dqv))
    duv = np.ma.array(duv, mask=np.isnan(duv))

    qv, uv, dqv, duv = ypu.correct_pa(qv, uv, dqv, duv, pa_obs=pa_obs, pa_ccw=False,
                                      in_pct=True, out_pct=True, in_deg=True)
    _gesd(qv, alpha=alpha, outliers=outliers, outliers_min=outliers_min)
    _gesd(uv, alpha=alpha, outliers=outliers, outliers_min=outliers_min)
    # _sigc(qv)
    # _sigc(uv)
    fullmask = qv.mask | uv.mask
    qv.mask = fullmask
    uv.mask = fullmask
    dqv.mask = fullmask
    duv.mask = fullmask
    # _mask_minmax(np.ma.array(qv, mask=np.isnan(qv)))
    # _mask_minmax(np.ma.array(uv, mask=np.isnan(uv)))
    # snrq = np.ma.array(snrq, mask=fullmask)
    # snru = np.ma.array(snru, mask=fullmask)

    # qv = _sigc(qv, sigma=sigma, stdfunc=stdfunc)
    # uv = _sigc(uv, sigma=sigma, stdfunc=stdfunc)
    q_w, dq_w = _wvg(qv, dqv)
    u_w, du_w = _wvg(uv, duv)
    return (q_w, u_w, dq_w, du_w, qv, uv)


def rtheta2xy(r, theta):
    theta = 180 - 2*np.array(theta)
    return r*np.cos(np.deg2rad(theta)), r*np.sin(np.deg2rad(theta))


basenames = [
    "SP_20190417", "SP_20190422", "UP_20180314", "UP_20180509",
    "Ceres_20200621", "Ceres_20201002", "Ceres_20201018", "Ceres_20210909", "Ceres_20211107",
    "Vesta_20191022", "Vesta_20191108", "Vesta_20191121", "Vesta_20191218", "Vesta_20200110", "Vesta_20200213"
]
rapmodes = ["min", "max", "minoe", "maxoe", 25, 30, 35, 40]

takahashi = dict(
    HD38563C=dict(p=[5.84, 3.59, 2.37], u=[73.8, 75.9, 74.8]),
    Elias9=dict(p=[np.nan]*3, u=[np.nan]*3),
    HDE283809=dict(p=[3.6, 2.38, 1.36], u=[58.7, 58.6, 58.8]),
    Elias29=dict(p=[2.70, 1.52, 1.08], u=[29.2, 31, 24.2]),
    Elias19=dict(p=[2.95, 1.8, 1.15], u=[36, 35, 35]),
    HD65583=dict(p=[np.sqrt(0.03**2 + (-0.06)**2), np.nan, np.sqrt(((-0.05)**2 + (-0.13)**2))],
                 u=[90 - 0.5*np.rad2deg(np.arctan2(-0.06, 0.03)),
                    np.nan,
                    90 - 0.5*np.rad2deg(np.arctan2(-0.05, -0.13))]),
    HD98281=dict(p=[np.sqrt((-0.05)**2 + (0.01**2)), np.nan, np.sqrt(((-0.04)**2 + (-0.05)**2))],
                 u=[90 - 0.5*np.rad2deg(np.arctan2(0.01, -0.05)),
                    np.nan,
                    90 - 0.5*np.rad2deg(np.arctan2(-0.05, -0.04))]),
    HD103095=dict(p=[np.sqrt((0.01)**2 + (0.02**2)), np.sqrt((0.03)**2 + ((-0.03)**2)), np.sqrt(((0.02)**2 + (-0.02)**2))],
                  u=[90 - 0.5*np.rad2deg(np.arctan2(0.02, 0.01)),
                     90 - 0.5*np.rad2deg(np.arctan2(0.03, -0.03)),
                     90 - 0.5*np.rad2deg(np.arctan2(0.02, -0.02))])
)
literature = dict(
    HD38563C=dict(p=[6.03, 3.68, 2.21], u=[71, 70, 78], dp=[0.1, 0.1, 0.55], du=[1, 1, 17]),
    Elias9=dict(p=[0.44, 0.41, 0.22], u=[145, 143, 146], dp=[0.02, 0.02, 0.02], du=[3, 1, 5]),
    HDE283809=dict(p=[3.81, 2.59, 1.71], u=[57, 58, 55], dp=[0.07, 0.07, 0.11], du=[1, 1, 3]),
    Elias29=dict(p=[2.43, 1.57, 1.07], u=[29, 27, 34], dp=[0.02, 0.02, 0.1], du=[1, 1, 4]),
    Elias19=dict(p=[3.06, 1.94, 1.36], u=[36, 35, 38], dp=[0.05, 0.03, 0.03], du=[1, 1, 1]),
)


/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:18: DeprecationWarning: `photutils.CircularAnnulus` is a deprecated alias for `photutils.aperture.CircularAnnulus` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAnnulus` to silence this warning.
  from photutils import CircularAnnulus, CircularAperture, EllipticalAperture
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:18: DeprecationWarning: `photutils.CircularAperture` is a deprecated alias for `photutils.aperture.CircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAperture` to silence this warning.
  from photutils import CircularAnnulus, CircularAperture, EllipticalAperture
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:18: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalApert

In [2]:
thresh_ksig = 10
minarea = 80

for basename in basenames:
    # if basename != basenames[2]:
    # continue
    name = basename + "_NHAO_NIC"

    npr = nic.NICPolReduc(
        name=basename,
        inputs=f"_original_32bit/{name}/rawdata/*.fits",
        #   verbose=1
    )
    df = npr.proc_lv4()  # to read the summ of lv4
    if df is None:
        continue

    # Initialize
    d4d = {_mode: dict() for _mode in rapmodes}
    for rapmode in rapmodes:
        d4d[rapmode] = dict(
            object=[], filt=[], midut=[], setid=[],
            snr_q=[], snr_u=[], dsnr_q=[], dsnr_u=[], pa=[],
            raps_q=[], raps_u=[],
            pos_q=[], pos_u=[],
            q=[], dtauq=[], u=[], dtauu=[], dq=[], du=[]
        )

    doit = [not Path(f"results/{basename}_{rapmode}.csv").exists() for rapmode in rapmodes]
    if any(doit):
        for objname, filt in itertools.product(df["OBJECT"].unique(), "JHK"):
            df_o_f = df.loc[(df["OBJECT"] == objname) & (df["FILTER"] == filt)]
            nid = len(df_o_f["SETID"].unique())
            gain = nic.GAIN[filt]
            rdn = nic.RDNOISE[filt]

            for i in range(1, nid+1):
                if len(df_o_f.loc[df_o_f["SETID"].isin([f"q{i:03d}", f"u{i:03d}"])]) != 8:
                    continue
                do_phot(df_o_f, i, d4d, rapmodes, objname, filt,
                        gain=1, rdn=0, flat_err=0.02, thresh_ksig=10, r_in=70, r_out=90)

        for rapmode in rapmodes:
            respath = Path(f"results/{basename}_{rapmode}.csv")
            dfp = pd.DataFrame.from_dict(d4d[rapmode])
            dfp["pol"] = np.sqrt(dfp["q"]**2 + dfp["u"]**2)
            dfp["dpol"] = np.sqrt((dfp["q"]*dfp["dq"])**2 + (dfp["u"]*dfp["du"])**2)/dfp["pol"]
            dfp["theta_p"] = 90 - np.rad2deg(0.5*np.arctan2(dfp["u"], dfp["q"]))
            dfp.to_csv(respath, index=False)



SKIP: Summary exists for lv4 at __logs/SP_20190417/SP_20190417_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/SP_20190422/SP_20190422_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/UP_20180314/UP_20180314_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/UP_20180509/UP_20180509_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Ceres_20200621/Ceres_20200621_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Ceres_20201002/Ceres_20201002_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Ceres_20201018/Ceres_20201018_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Ceres_20210909/Ceres_20210909_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Ceres_20211107/Ceres_20211107_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Vesta_20191022/Vesta_20191022_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Vesta_20191108/Vesta_20191108_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/Vesta_20191121/Vesta_20191121_summ_lv4.csv
SKIP: Summary exists for lv4 at __logs/V

## Code below are archival --- do not run them

In [3]:
for basename in basenames:
    npr = nic.NICPolReduc(
        name=basename,
        inputs=None,  # dummy
    )
    df = npr.proc_lv4()  # to read the summ of lv4

    for rapmode in rapmodes:
        respath = Path(f"results/{basename}_{rapmode}.csv")
        print(respath)
        if not respath.exists():
            continue
        dfp = pd.read_csv(respath)

        for objname, mk, mklit in zip(dfp.object.unique(), ["1", "+", "3", "v"], ["X", "P", "o"]):
            if objname == "sky":
                continue
            # if objname != "HD65583":
            #     continue
            fig, axs = plt.subplots(1, 1, figsize=(9, 6), sharex=False, sharey=False, gridspec_kw=None)

            for i_filt, (filt, col) in enumerate(zip("JHK", "bgr")):
                # remove if
                # (1) minimum SNR < 50 (faint object) or
                # (2) delta-SNR > minimum SNR (abrupt sky change) or
                # (3) nominal error in q or u > 0.5
                mask = ((dfp["filt"] == filt) & (dfp["object"] == objname)
                        & (dfp["snr_q"] > 50) & (dfp["snr_u"] > 50)
                        & (dfp["dsnr_q"] < (dfp["snr_q"])) & (dfp["dsnr_u"] < (dfp["snr_u"]))
                        & (dfp["dq"] < 0.5) & (dfp["du"] < 0.5))
                _dfp = dfp[mask]

                q, u, dq, du, qv, uv = qupa(_dfp["q"].to_numpy(), _dfp["u"].to_numpy(),
                                            _dfp["dq"].to_numpy(), _dfp["du"].to_numpy(),
                                            _dfp["pa"].to_numpy(), alpha=0.05, outliers=0.5, outliers_min=1)
                sqrt_n = np.sqrt(qv.compressed().size)
                sq = np.ma.std(qv, ddof=1) / sqrt_n if qv.compressed().size > 1 else np.nan
                su = np.ma.std(uv, ddof=1) / sqrt_n if qv.compressed().size > 1 else np.nan
                # (objname, filt, qv)
                p0 = np.sqrt(q**2+u**2)
                dp0 = np.sqrt((q*sq)**2 + (u*su)**2)/p0
                nuse = qv.size - (qv.mask | uv.mask).sum()
                print(f"{objname} ({filt}): {p0:.2f}±{dp0:.2f} ({nuse}/{qv.size}, snr={p0/dp0:.0f})")

                kw = dict(color=col, marker=".")
                axs.errorbar(qv.compressed(), uv.compressed(),
                             xerr=_dfp["dq"][~qv.mask], yerr=_dfp["du"][~uv.mask],
                             capsize=0, elinewidth=0.5, ls="", alpha=0.4, **kw)
                if not np.isnan(q) and not np.isnan(u):
                    # quap = EllipticalAperture([q, u], a=dq, b=du, theta=0)
                    # quap.plot(axs, color=col)
                    if sq > 0 and su > 0:
                        quap = EllipticalAperture([q, u], a=sq, b=su, theta=0)
                        quap.plot(axs, color=col)
                axs.scatter(
                    np.nan, np.nan, **kw,
                    label=(f"{objname} ({filt})\nP = {p0:.2f}±{dp0:.2f}\n"
                           + r"use=$\,\frac{{{}}}{{{}}}$, $\frac{{P}}{{ΔP}}=${:.0f}".format(nuse, qv.size, p0/dp0)
                           + "\nq={:+.2f}±{:.2f}%\nu={:+.2f}±{:.2f}%".format(q, sq, u, su)
                           )
                )

                for _q, _u, _id in zip(qv.compressed(), uv.compressed(), _dfp["setid"].to_numpy()[~qv.mask]):
                    # txt = "({:.1f}, {:.1f}) = {:.1f}\n({:.1f}, {:.1f}) = {:.1f}".format(
                    #     row["dxq"], row["dyq"], row["drq"], row["dxu"], row["dyu"], row["dru"])
                    # txt = row["setid"]
                    axs.text(_q, _u, _id, color=col, fontsize=8)

                if objname in takahashi:
                    axs.plot(*rtheta2xy([takahashi[objname]["p"][i_filt]], [takahashi[objname]["u"][i_filt]]),
                             marker="x", color=col, ms=15)
                if objname in literature:
                    axs.scatter(*rtheta2xy([literature[objname]["p"][i_filt]], [literature[objname]["u"][i_filt]]),
                                s=500, color=col, marker="o", alpha=0.5)

            axs.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=10)
            axs.set_aspect("equal")
            axs.axhline(0, lw=2, color="k")
            axs.axvline(0, lw=2, color="k")
            delta = np.max((np.ptp(axs.get_xlim())/2, np.ptp(axs.get_ylim())/2))

            axs.set(
                xlabel="q_PAcorr [%]",
                ylabel="u_PAcorr [%]",
                xlim=(np.mean(axs.get_xlim()) - delta, np.mean(axs.get_xlim()) + delta),
                ylim=(np.mean(axs.get_ylim()) - delta, np.mean(axs.get_ylim()) + delta),
            )

            plt.tight_layout()
            fig.align_ylabels(axs)
            fig.align_xlabels(axs)
            plt.savefig(f"results/figs/{basename}_{objname}_{rapmode}.png", dpi=300)
            plt.close("all")



SKIP: Summary exists for lv4 at __logs/SP_20190417/SP_20190417_summ_lv4.csv
results/SP_20190417_min.csv
HDE283809 (J): 3.61±0.06 (7/7, snr=61)
HDE283809 (H): nan±nan (0/0, snr=nan)
HDE283809 (K): 1.46±0.05 (7/7, snr=31)


/Users/ysbach/miniforge3/lib/python3.10/site-packages/numpy/ma/extras.py:649: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt,
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:212: RuntimeWarning: divide by zero encountered in double_scalars
  1/np.sqrt(np.ma.sum(1/dx.compressed()**2)))


Elias29 (J): 2.50±0.05 (5/5, snr=52)
Elias29 (H): 1.52±0.03 (4/5, snr=56)
Elias29 (K): 0.88±0.03 (5/5, snr=31)
Elias19 (J): 3.05±0.03 (20/20, snr=98)
Elias19 (H): nan±nan (0/0, snr=nan)
Elias19 (K): 1.20±0.04 (20/20, snr=32)
results/SP_20190417_max.csv
HDE283809 (J): 3.62±0.06 (7/7, snr=60)
HDE283809 (H): nan±nan (0/0, snr=nan)
HDE283809 (K): 1.46±0.05 (7/7, snr=31)


/Users/ysbach/miniforge3/lib/python3.10/site-packages/numpy/ma/extras.py:649: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt,
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:212: RuntimeWarning: divide by zero encountered in double_scalars
  1/np.sqrt(np.ma.sum(1/dx.compressed()**2)))


Elias29 (J): 2.47±0.06 (4/5, snr=45)
Elias29 (H): 1.52±0.03 (4/5, snr=49)
Elias29 (K): 0.89±0.03 (5/5, snr=28)
Elias19 (J): 3.06±0.03 (20/20, snr=95)
Elias19 (H): nan±nan (0/0, snr=nan)
Elias19 (K): 1.20±0.04 (20/20, snr=32)
results/SP_20190417_minoe.csv
HDE283809 (J): 3.61±0.06 (7/7, snr=62)
HDE283809 (H): nan±nan (0/0, snr=nan)
HDE283809 (K): 1.46±0.05 (7/7, snr=31)


...

FS12 (J): nan±nan (0/0, snr=nan)
FS12 (H): nan±nan (0/0, snr=nan)
FS12 (K): nan±nan (0/0, snr=nan)
results/Vesta_20191121_35.csv
Vesta (J): 0.64±0.03 (158/160, snr=24)
Vesta (H): 0.71±0.04 (106/109, snr=18)
Vesta (K): 0.92±0.05 (157/160, snr=20)


/Users/ysbach/miniforge3/lib/python3.10/site-packages/numpy/ma/extras.py:649: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt,
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:212: RuntimeWarning: divide by zero encountered in double_scalars
  1/np.sqrt(np.ma.sum(1/dx.compressed()**2)))


FS12 (J): nan±nan (0/0, snr=nan)
FS12 (H): nan±nan (0/0, snr=nan)
FS12 (K): nan±nan (0/0, snr=nan)
results/Vesta_20191121_40.csv
Vesta (J): 0.64±0.03 (158/160, snr=23)
Vesta (H): 0.71±0.04 (106/109, snr=18)
Vesta (K): 0.92±0.05 (156/160, snr=20)


/Users/ysbach/miniforge3/lib/python3.10/site-packages/numpy/ma/extras.py:649: RuntimeWarning: invalid value encountered in double_scalars
  avg = np.multiply(a, wgt,
/var/folders/b1/mdnx6bfn6_v7p70xc3hpc9vh0000gn/T/ipykernel_76823/2745043712.py:212: RuntimeWarning: divide by zero encountered in double_scalars
  1/np.sqrt(np.ma.sum(1/dx.compressed()**2)))


FS12 (J): nan±nan (0/0, snr=nan)
FS12 (H): nan±nan (0/0, snr=nan)
FS12 (K): nan±nan (0/0, snr=nan)
SKIP: Summary exists for lv4 at __logs/Vesta_20191218/Vesta_20191218_summ_lv4.csv
results/Vesta_20191218_min.csv
Vesta (J): 0.40±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.03 (67/67, snr=16)
Vesta (K): 0.66±0.04 (67/67, snr=16)
results/Vesta_20191218_max.csv
Vesta (J): 0.41±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.03 (67/67, snr=16)
Vesta (K): 0.66±0.04 (67/67, snr=16)
results/Vesta_20191218_minoe.csv
Vesta (J): 0.41±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.03 (67/67, snr=16)
Vesta (K): 0.66±0.04 (67/67, snr=16)
results/Vesta_20191218_maxoe.csv
Vesta (J): 0.41±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.03 (67/67, snr=16)
Vesta (K): 0.66±0.04 (67/67, snr=16)
results/Vesta_20191218_25.csv
Vesta (J): 0.41±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.04 (67/67, snr=16)
Vesta (K): 0.66±0.04 (67/67, snr=16)
results/Vesta_20191218_30.csv
Vesta (J): 0.41±0.03 (67/67, snr=12)
Vesta (H): 0.55±0.03 (67/67, snr=16)
Ve

In [ ]:
name = "SP_20190417_NHAO_NIC"
basename = "SP_20190417"
npr = nic.NICPolReduc(
    name=basename,
    inputs=f"_original_32bit/{name}/rawdata/*.fits",
    #   verbose=1
)
df = npr.proc_lv4()  # to read the summ of lv4

const_rap = False

d4d = dict(
    object=[], filt=[], setid=[], snr_q=[], snr_u=[], pa=[],
    raps_q=[], raps_u=[],
    pos_q=[], pos_u=[],
    q=[], dtauq=[], u=[], dtauu=[]
)

thresh_ksig = 10
minarea = 80

objname = "Elias19"
filt = "K"
qu = "q"
i = 8

df_o_f = df.loc[(df["OBJECT"] == objname) & (df["FILTER"] == filt)]
nid = len(df_o_f["SETID"].unique())
gain = nic.GAIN[filt]
rdn = nic.RDNOISE[filt]

_df = df_o_f.loc[df_o_f["SETID"] == f"{qu}{i:03d}"].copy()
# order = HWP1 o, HWP2 o, HWP1 e, HWP2 e (a, c, b, d)
im_acbd = yfu.load_ccds(list(_df["file"]), ccddata=False, trimsec=10)
err_acbd = [yfu.errormap(im, gain_epadu=gain, rdnoise_electron=rdn, flat_err=0.02) for im in im_acbd]
pos_acbd = [find_pos(im, thresh_ksig=thresh_ksig) for im in im_acbd]
rsnr_acbd = [find_rsnr(im, pos[0], sky_an=CircularAnnulus(pos[0], 70, 90), err=err, const=const_rap)[0]
             for im, pos, err in zip(im_acbd, pos_acbd, err_acbd)]
print([res for res in rsnr_acbd])
r_apmax = max(rsnr_acbd)
rap_o = min(rsnr_acbd[: 2])
rap_e = min(rsnr_acbd[2: ])

for k, (_, row) in enumerate(_df.iterrows()):
    pos = pos_acbd[k][0]
    # obj = obj_o if k < 2 else obj_e
    r_ap = rap_o if k < 2 else rap_e
    ap, an = ypu.circ_ap_an(pos, r_ap, r_in=70, r_out=90)
    phot = ypu.apphot_annulus(im_acbd[k], ap, an, error=err_acbd[k])
    # pos = find_pos(im, thresh_ksig=10)
    # r_ap, phot, ap, an = find_rap(img0, pos, r_init=40, r_in=70, r_out=90, error=err, const=const_rap)
    print(objname, filt, qu, i, "acbd"[k], pos, phot["snr"][0], r_ap)

    aps.append(ap)
    ans.append(an)
    raps.append(r_ap)
    poss.append(pos)
    acbd.append(phot["source_sum"].iloc[0])
    snrs.append(phot["snr"].iloc[0])
    pas.append(row["PA"])
quval = calcq(acbd[0], acbd[2], acbd[1], acbd[3])
dtau = 0.5*np.log((acbd[0]*acbd[3])/(acbd[1]*acbd[2]))

fig, axs = plt.subplots(1, 4, figsize=(8, 5), sharex=True, sharey=True, gridspec_kw=None)

for aa, im, pos, r_ap in zip(axs.flatten(), im_acbd, pos_acbd, [rap_o, rap_o, rap_e, rap_e]):
    yvu.norm_imshow(aa, im, stretch="asinh")
    ap, an = ypu.circ_ap_an(pos[0], r_ap, r_in=70, r_out=90)
    ap.plot(aa)
    ap, an = ypu.circ_ap_an(pos[0], r_apmax, r_in=70, r_out=90)
    ap.plot(aa, color="r")


plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 5), sharex=False, sharey=False, gridspec_kw=None)
aps = [CircularAperture(pos_acbd[0][0], r=_r) for _r in np.arange(5, 50)]
di = np.array([sigma_clipped_stats(ypu.annul2values(im_acbd[0], CircularAnnulus(pos_acbd[0][0], r_in=_r-0.5, r_out=_r+0.5)))[1]
               for _r in np.arange(5, 50)])
phot_all = ypu.apphot_annulus(im_acbd[0], aps, CircularAnnulus(pos_acbd[0][0], 70, 90), error=err_acbd[0])
print(di/gain > phot_all["ssky"][0]**2)
print(np.ediff1d(phot_all["aperture_sum"])/np.ediff1d(phot_all["aparea"])/gain > phot_all["ssky"][0]**2)

dist, prof = ypu.radprof_pix(im_acbd[0], pos_acbd[0][0], rmax=100, sort_dist=True)
axs.plot(dist, prof, ".")
axs.plot(dist, csaps.csaps(dist, prof, dist, smooth=0.01), "r")

# for k in np.arange(0.1, 1, 0.1):
#     axs.plot(dist, csaps.csaps(dist, prof, dist, smooth=k), label=f"{k}")
axs.legend()
axs.set(yscale="log")
plt.tight_layout()
plt.show()


In [ ]:
thresh_ksig = 10
minarea = 80

for basename in basenames:
    # if basename != basenames[2]:
    # continue
    name = basename + "_NHAO_NIC"

    npr = nic.NICPolReduc(
        name=basename,
        inputs=f"_original_32bit/{name}/rawdata/*.fits",
        #   verbose=1
    )
    df = npr.proc_lv4()  # to read the summ of lv4
    if df is None:
        continue

    d4d = {_mode: dict() for _mode in rapmodes}

    for rapmode in rapmodes:
        respath = Path(f"results/{basename}_{rapmode}.csv")

        const_rap = rapmode if isinstance(rapmode, (float, int)) else False

        if not respath.exists():
            d4d = dict(
                object=[], filt=[], midut=[], setid=[], snr_q=[], snr_u=[], pa=[],
                raps_q=[], raps_u=[],
                pos_q=[], pos_u=[],
                q=[], dtauq=[], u=[], dtauu=[], dq=[], du=[]
            )
            for objname, filt in itertools.product(df["OBJECT"].unique(), "JHK"):
                df_o_f = df.loc[(df["OBJECT"] == objname) & (df["FILTER"] == filt)]
                nid = len(df_o_f["SETID"].unique())
                gain = nic.GAIN[filt]
                rdn = nic.RDNOISE[filt]

                for i in range(1, nid+1):
                    _df = df_o_f.loc[df_o_f["SETID"].isin([f"q{i:03d}", f"u{i:03d}"])]
                    if len(_df) != 8:
                        continue

                    pas = []
                    uts = []
                    acbds = {_mode: dict() for _mode in rapmodes}
                    dacbds = {_mode: dict() for _mode in rapmodes}
                    for qu in "qu":
                        _df = df_o_f.loc[df_o_f["SETID"] == f"{qu}{i:03d}"].copy()
                        # order = HWP1 o, HWP2 o, HWP1 e, HWP2 e (a, c, b, d)
                        im_acbd = yfu.load_ccds(list(_df["file"]), ccddata=False, trimsec=10)
                        err_acbd = [yfu.errormap(im, gain_epadu=gain, rdnoise_electron=rdn, flat_err=0.02) for im in im_acbd]
                        pos_acbd = [find_pos(im, thresh_ksig=thresh_ksig) for im in im_acbd]
                        rsnr_acbd = [find_rsnr(im, pos[0], sky_an=CircularAnnulus(pos[0], 70, 90), err=err, const=const_rap)[0]
                                     for im, pos, err in zip(im_acbd, pos_acbd, err_acbd)]
                        print([res for res in rsnr_acbd])

                        if rapmode == "max":
                            r_ap = np.max(rsnr_acbd)
                        elif rapmode == "min":
                            r_ap = np.min(rsnr_acbd)
                        elif rapmode == "maxoe":
                            rap_o = max(rsnr_acbd[:2])
                            rap_e = max(rsnr_acbd[2:])
                        elif rapmode == "minoe":
                            rap_o = min(rsnr_acbd[:2])
                            rap_e = min(rsnr_acbd[2:])
                        else:
                            r_ap = const_rap

                        acbd = []
                        dacbd = []
                        snrs = []
                        raps = []
                        poss = []
                        aps = []
                        ans = []

                        for k, (_, row) in enumerate(_df.iterrows()):
                            pos = pos_acbd[k][0]
                            # obj = obj_o if k < 2 else obj_e
                            if rapmode in ["maxoe", "minoe"]:
                                r_ap = rap_o if k < 2 else rap_e

                            ap, an = ypu.circ_ap_an(pos, r_ap, r_in=70, r_out=90)
                            phot = ypu.apphot_annulus(im_acbd[k], ap, an, error=err_acbd[k])
                            # pos = find_pos(im, thresh_ksig=10)
                            # r_ap, phot, ap, an = find_rap(img0, pos, r_init=40, r_in=70, r_out=90, error=err, const=const_rap)

                            aps.append(ap)
                            ans.append(an)
                            raps.append(r_ap)
                            poss.append([float(f"{_p:.3f}") for _p in pos])
                            acbd.append(phot["source_sum"].iloc[0])
                            dacbd.append(phot["source_sum_err"].iloc[0])
                            snrs.append(phot["snr"].iloc[0])
                            pas.append(row["PA"])
                            uts.append((Time(row["DATE-OBS"], format="isot") + row["EXPTIME"]*units.s/2).jd)

                            print(objname, filt, qu, i, "acbd"[k],
                                  [float(f"{_p:.3f}") for _p in pos], phot["snr"][0], r_ap)

                        dtau = 0.5*np.log((acbd[0]*acbd[3])/(acbd[1]*acbd[2]))
                        d4d[f"dtau{qu}"].append(dtau)
                        d4d[f"snr_{qu}"].append(np.mean(snrs))
                        d4d[f"raps_{qu}"].append(raps)
                        d4d[f"pos_{qu}"].append(poss)
                        acbds[qu] = acbd
                        dacbds[qu] = dacbd
                    q, u, dq, du = ypu.calc_qu_4set(o_000=acbds["q"][0], e_000=acbds["q"][2],
                                                    o_450=acbds["q"][1], e_450=acbds["q"][3],
                                                    o_225=acbds["u"][0], e_225=acbds["u"][2],
                                                    o_675=acbds["u"][1], e_675=acbds["u"][3],
                                                    do_000=dacbds["q"][0], de_000=dacbds["q"][2],
                                                    do_450=dacbds["q"][1], de_450=dacbds["q"][3],
                                                    do_225=dacbds["u"][0], de_225=dacbds["u"][2],
                                                    do_675=dacbds["u"][1], de_675=dacbds["u"][3], out_pct=True)

                    d4d["setid"].append(i)
                    d4d["filt"].append(filt)
                    d4d["object"].append(objname)
                    d4d["pa"].append(np.mean(pas))
                    d4d["midut"].append(Time(np.mean(uts), format="jd").isot)
                    d4d["q"].append(q)
                    d4d["u"].append(u)
                    d4d["dq"].append(dq)
                    d4d["du"].append(du)

                    # print("{:.3f}%, {:.3f}%, qu_im2-qu_fin={:.3f}%    {:.0f}".format(p2, pf, dp, np.mean(snrs)))
            dfp = pd.DataFrame.from_dict(d4d)
            dfp["pol"] = np.sqrt(dfp["q"]**2 + dfp["u"]**2)
            dfp["dpol"] = np.sqrt((dfp["q"]*dfp["dq"])**2 + (dfp["u"]*dfp["du"])**2)/dfp["pol"]
            dfp["theta_p"] = 90 - np.rad2deg(0.5*np.arctan2(dfp["u"], dfp["q"]))
            dfp.to_csv(respath, index=False)


In [ ]:

# basename = basenames[0]
# name = basename + "_NHAO_NIC"

# npr = nic.NICPolReduc(
#     name=basename,
#     inputs=f"_original_32bit/{name}/rawdata/*.fits",
#     #   verbose=1
# )
# df = npr.proc_lv4()  # to read the summ of lv4

# for rapmode in rapmodes:
#     respath = Path(f"results/{basename}_{rapmode}.csv")

#     const_rap = rapmode if isinstance(rapmode, (float, int)) else False
#     thresh_ksig = 10
#     minarea = 80

#     d4d = dict(
#         object=[], filt=[], setid=[], snr_q=[], snr_u=[], pa=[],
#         raps_q=[], raps_u=[],
#         pos_q=[], pos_u=[],
#         q=[], dtauq=[], u=[], dtauu=[], dq=[], du=[]
#     )

#     for objname, filt in itertools.product(df["OBJECT"].unique(), "JHK"):
#         df_o_f = df.loc[(df["OBJECT"] == objname) & (df["FILTER"] == filt)]
#         nid = len(df_o_f["SETID"].unique())
#         gain = nic.GAIN[filt]
#         rdn = nic.RDNOISE[filt]

#         for i in range(1, nid+1):
#             _df = df_o_f.loc[df_o_f["SETID"].isin([f"q{i:03d}", f"u{i:03d}"])]
#             if len(_df) != 8:
#                 continue

#             pas = []
#             acbds = {}
#             dacbds = {}
#             for qu in "qu":
#                 _df = df_o_f.loc[df_o_f["SETID"] == f"{qu}{i:03d}"].copy()
#                 # order = HWP1 o, HWP2 o, HWP1 e, HWP2 e (a, c, b, d)
#                 im_acbd = yfu.load_ccds(list(_df["file"]), ccddata=False, trimsec=10)
#                 err_acbd = [yfu.errormap(im, gain_epadu=gain, rdnoise_electron=rdn, flat_err=0.02) for im in im_acbd]
#                 pos_acbd = [find_pos(im, thresh_ksig=thresh_ksig) for im in im_acbd]
#                 rsnr_acbd = [find_rsnr(im, pos[0], sky_an=CircularAnnulus(pos[0], 70, 90), err=err, const=const_rap)[0]
#                              for im, pos, err in zip(im_acbd, pos_acbd, err_acbd)]
#                 print([res for res in rsnr_acbd])

#                 if rapmode == "max":
#                     r_ap = np.max(rsnr_acbd)
#                 elif rapmode == "min":
#                     r_ap = np.min(rsnr_acbd)
#                 elif rapmode == "maxoe":
#                     rap_o = max(rsnr_acbd[:2])
#                     rap_e = max(rsnr_acbd[2:])
#                 elif rapmode == "minoe":
#                     rap_o = min(rsnr_acbd[:2])
#                     rap_e = min(rsnr_acbd[2:])
#                 else:
#                     r_ap = const_rap

#                 acbd = []
#                 dacbd = []
#                 snrs = []
#                 raps = []
#                 poss = []
#                 aps = []
#                 ans = []

#                 for k, (_, row) in enumerate(_df.iterrows()):
#                     pos = pos_acbd[k][0]
#                     # obj = obj_o if k < 2 else obj_e
#                     if rapmode in ["maxoe", "minoe"]:
#                         r_ap = rap_o if k < 2 else rap_e

#                     ap, an = ypu.circ_ap_an(pos, r_ap, r_in=70, r_out=90)
#                     phot = ypu.apphot_annulus(im_acbd[k], ap, an, error=err_acbd[k])
#                     # pos = find_pos(im, thresh_ksig=10)
#                     # r_ap, phot, ap, an = find_rap(img0, pos, r_init=40, r_in=70, r_out=90, error=err, const=const_rap)

#                     aps.append(ap)
#                     ans.append(an)
#                     raps.append(r_ap)
#                     poss.append([float(f"{_p:.3f}") for _p in pos])
#                     acbd.append(phot["source_sum"].iloc[0])
#                     dacbd.append(phot["source_sum_err"].iloc[0])
#                     snrs.append(phot["snr"].iloc[0])
#                     pas.append(row["PA"])
#                     print(objname, filt, qu, i, "acbd"[k], [float(f"{_p:.3f}") for _p in pos], phot["snr"][0], r_ap)
#                 dtau = 0.5*np.log((acbd[0]*acbd[3])/(acbd[1]*acbd[2]))

#                 d4d[f"dtau{qu}"].append(dtau)
#                 d4d[f"snr_{qu}"].append(np.mean(snrs))
#                 d4d[f"raps_{qu}"].append(raps)
#                 d4d[f"pos_{qu}"].append(poss)
#                 acbds[qu] = acbd
#                 dacbds[qu] = dacbd
#             q, u, dq, du = ypu.calc_qu_4set(o_000=acbds["q"][0], e_000=acbds["q"][2],
#                                             o_450=acbds["q"][1], e_450=acbds["q"][3],
#                                             o_225=acbds["u"][0], e_225=acbds["u"][2],
#                                             o_675=acbds["u"][1], e_675=acbds["u"][3],
#                                             do_000=dacbds["q"][0], de_000=dacbds["q"][2],
#                                             do_450=dacbds["q"][1], de_450=dacbds["q"][3],
#                                             do_225=dacbds["u"][0], de_225=dacbds["u"][2],
#                                             do_675=dacbds["u"][1], de_675=dacbds["u"][3], out_pct=True)

#             d4d["setid"].append(i)
#             d4d["filt"].append(filt)
#             d4d["object"].append(objname)
#             d4d["pa"].append(np.mean(pas))
#             d4d["q"].append(q)
#             d4d["u"].append(u)
#             d4d["dq"].append(dq)
#             d4d["du"].append(du)

#             # print("{:.3f}%, {:.3f}%, qu_im2-qu_fin={:.3f}%    {:.0f}".format(p2, pf, dp, np.mean(snrs)))
#     dfp = pd.DataFrame.from_dict(d4d)
#     dfp["pol"] = np.sqrt(dfp["q"]**2 + dfp["u"]**2)
#     dfp["dpol"] = np.sqrt((dfp["q"]*dfp["dq"])**2 + (dfp["u"]*dfp["du"])**2)/dfp["pol"]
#     dfp["theta_p"] = 90 - np.rad2deg(0.5*np.arctan2(dfp["u"], dfp["q"]))
#     dfp.to_csv(respath, index=False)
# # %%

# ypu.calc_qu_4set(o_000=acbds["q"][0], e_000=acbds["q"][2],
#                  o_450=acbds["q"][1], e_450=acbds["q"][3],
#                  o_225=acbds["u"][0], e_225=acbds["u"][2],
#                  o_675=acbds["u"][1], e_675=acbds["u"][3],
#                  do_000=dacbds["q"][0], de_000=dacbds["q"][2],
#                  do_450=dacbds["q"][1], de_450=dacbds["q"][3],
#                  do_225=dacbds["u"][0], de_225=dacbds["u"][2],
#                  do_675=dacbds["u"][1], de_675=dacbds["u"][3])
